# Building an ensemble of models from the Cell fate model

This notebook is adapted from the one made available with the article Synthesis and Simulation of Ensembles of
Boolean Networks for Cell Fate Decision by Chevalier et al. (CMSB2020 proceedings) 
https://hal.archives-ouvertes.fr/hal-02898849/file/cmsb2020.pdf

It shows how, starting from an interaction graph and a list of fixed points, we can build an ensemble of boolean models compatible with these constraints.

In [1]:
import bonesis
import ginsim
import biolqm
import mpbn
import maboss
import pandas as pd
from colomoto_jupyter import tabulate
import os

These are possible settings for the generation of the ensemble by Bonesis.

In [2]:
exact_pkn = True
maxclause = 32
exclude_cyclic = True
bundle_prefix = "bundle-{}pkn{}-{}".format("exact" if exact_pkn else "any", maxclause,
        "nocyclic-" if exclude_cyclic else "")
limit = 2000
threads = 4

### Loading MaBoSS model

In [3]:
mbs_model = maboss.load("CellFateModel.bnd", "CellFateModel.cfg")

### Computing list of fixed points 

In [4]:
blqm_model = maboss.to_biolqm(mbs_model)
fps = biolqm.fixpoints(blqm_model)
tabulate(fps)

,FASL,TNF,TNFR,FADD,DISC_TNF,DISC_FAS,CASP8,RIP1,cIAP,RIP1ub,RIP1K,IKK,CASP3,NFkB,cFLIP,BCL2,BAX,mROS,MPT,ROS,ATP,MOMP,SMAC,mcIAP,Cyt_c,mXIAP,XIAP,apoptosome,NonACD,Apoptosis,Survival
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,1,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,1,1,0,1,0,0,1,0,1,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,1,0,0,0,1,0,0
5,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
7,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,1,1,0,1,0,0,1,0,1,0
8,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,1,0,1,1,0,1,0,0,0,1,0,0
9,0,1,1,0,0,0,0,1,1,1,1,1,0,1,1,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,1


## Architecture

We take the influence graph of the original model as constraint for the synthesis.
We impose that all identified BNs have the exact same influence graph.
As some nodes have high in-degree, we limit the number of clauses in Boolean functions.

In [5]:
bn = mpbn.MPBooleanNetwork(maboss.to_minibn(mbs_model))

In [6]:
dom = bonesis.InfluenceGraph.from_ginsim(biolqm.to_ginsim(maboss.to_biolqm(mbs_model)), exact=exact_pkn, maxclause=maxclause)

## Input Data

### Initial condition

Our initial states can be have `TNF` active or not, `ATP`, `FADD` and `cIAP` active, all other nodes are inactive.

In [7]:
init = bn.zero()
# del free nodes
del init["TNF"]
init["FADD"] = 1
init["ATP"] = 1
init["cIAP"] = 1
init

{'FASL': 0,
 'TNFR': 0,
 'FADD': 1,
 'DISC_TNF': 0,
 'DISC_FAS': 0,
 'CASP8': 0,
 'RIP1': 0,
 'cIAP': 1,
 'RIP1ub': 0,
 'RIP1K': 0,
 'IKK': 0,
 'CASP3': 0,
 'NFkB': 0,
 'cFLIP': 0,
 'BCL2': 0,
 'BAX': 0,
 'mROS': 0,
 'MPT': 0,
 'ROS': 0,
 'ATP': 1,
 'MOMP': 0,
 'SMAC': 0,
 'mcIAP': 0,
 'Cyt_c': 0,
 'mXIAP': 0,
 'XIAP': 0,
 'apoptosome': 0,
 'NonACD': 0,
 'Apoptosis': 0,
 'Survival': 0}

### Phenotypes

In [8]:
outputs = ["Survival", "NonACD","Apoptosis"]
def make_marker(*markers):
    for n in markers: # ensure nodes exist
        assert n in bn
    return {n: 1 if n in markers else 0 for n in outputs}
phenotypes = {
"HS": make_marker(),
"Apoptosis": make_marker("Apoptosis"),
"NonACD": make_marker("NonACD"),
"Survival": make_marker("Survival"),
}
pd.DataFrame.from_dict(phenotypes, orient="index").fillna('')

,Survival,NonACD,Apoptosis
HS,0,0,0
Apoptosis,0,0,1
NonACD,0,1,0
Survival,1,0,0


## Synthesis

In [9]:
data = {"init": init}
data.update(phenotypes)

In [10]:
bo = bonesis.BoNesis(dom, data)
bo.settings["parallel"] = threads

In [11]:
def has_cyclic(bn):
    mbn = mpbn.MPBooleanNetwork(bn)
    for a in mbn.attractors():
        if "*" in a.values():
            return True
    return False

In [12]:
def build_ensemble(bo, name, limit):
    folder = f"{bundle_prefix}{name}"
    if not os.path.exists(folder):
        os.mkdir(folder)
    print(f"Building ensemble of size {limit} in folder {folder}")
    mylimit = 2*limit
    view = bo.diverse_boolean_networks(limit=mylimit, skip_supersets=True)
    n = 0
    for i, bn in enumerate(view):
        if exclude_cyclic and has_cyclic(bn):
            print("HAS CYCLIC ATTRACTORS, IGNORING")
            continue
        with open(f"{folder}/bn{i}.bnet", "w") as fp:
            fp.write(bn.source())
        n += 1
        if n == limit:
            break

### Wild-type constraints

Each phenotype has at least one matching fixed point, which is reachable from at least one initial condition.

In [13]:
wt_phenotypes = ["HS", "Apoptosis", "NonACD", "Survival"]

In [14]:
for ph in wt_phenotypes:
    # there is one configuration matching with init
    # which can reach a configuration being a fixed point and matching with ph
    +bo.obs("init") >= bo.fixed(+bo.obs(ph))

In [15]:
%time build_ensemble(bo, "v0", limit)